Имея решение `1d`  задачи для двух жидкостей, можем получить `2d` решение

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import utils as u
import scipy


from IPython import display

In [ ]:
import torch
import ctypes
ctypes.cdll.LoadLibrary('caffe2_nvrtc.dll')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
dtype = torch.float64
device

In [ ]:
def two_dim_index_to_one(i: int, j: int, ny: int) -> int:
    return ny * i + j
def one_d_index_to_two(one_d: int, ny: int):
    i = int(one_d / ny)
    j = one_d % ny
    return i, j

In [ ]:
nx = 200
ny = 200
k = 1e-1 * 1.987e-13  # 1 darcy to m^2
dx = 3 # m
dy = 3 # m
phi = 0.4
p_0 = 150 * 10 ** 5  # psi to Pa
d = 10  # m
dt = 30  # s
s_0 = 0.4
s_b = 0.1
p = torch.ones((nx*ny, 1), device=device, dtype=dtype) * p_0
s_o = torch.ones((nx*ny, 1), device=device, dtype=dtype) * s_0
s_w = torch.ones((nx*ny, 1), device=device, dtype=dtype)* (1 - s_0)

Перейдём к сжимаемости для [rock](https://www.sciencedirect.com/topics/engineering/formation-compressibility), [water and oil](http://www.fekete.com/san/webhelp/feketeharmony/harmony_webhelp/content/html_files/reference_material/general_concepts/reservoir_properties.htm)

In [ ]:
c_w = 1e-6 # # Pa^-1
c_o = 1e-6 # # Pa^-1
c_r = 3e-6 #  Pa^-1 

Также различаются вязкости [oil](https://petrowiki.org/Oil_viscosity) и [water](http://fekete.com/SAN/TheoryAndEquations/HarmonyTheoryEquations/Content/HTML_Files/Reference_Material/General_Concepts/Reservoir_Fluid_Properties.htm)

In [ ]:
B_o = 1
B_w = 1
mu_o = 15 / 1000 # cp to Pa * s
mu_w = 1 / 1000 # cp to Pa * s

Такс, вводим гиперпараметры для относитльной проницаемости

In [ ]:
l_w = 2.
l_o = 2.
s_wir = 0.2
s_wor = 0.8
k_rwr = 0.1
k_rot = 1.
e_w = 1.
e_o = 1.
t_w = 2.
t_o = 2.

Запихнём всю эту информацию в специальный класс

In [ ]:
import res_properties
import imp
imp.reload(res_properties)

In [ ]:
prop = res_properties.Properties(nx=nx, ny=ny, k=k, dx=dx, dy=dy, phi=phi, p_0=p_0, d=d, dt=dt, s_0=s_0,
                 c_w=c_w, c_o=c_w, c_r=c_r, mu_w=mu_w, mu_o=mu_o, b_o=B_o, b_w=B_w, l_w=l_w, l_o=l_o,
                 s_wir=s_wir, s_wor=s_wor, k_rwr=k_rwr, k_rot=k_rot, e_w=e_w, e_o=e_o, t_w=t_w, t_o=t_o)

In [ ]:
ss = np.linspace(0, 1, 200)
k_rels_w = [prop.k_rel_w(1-s) for s in ss]
k_rels_o = [prop.k_rel_o(s) for s in ss]
plt.plot(ss, k_rels_w, label='water')
plt.plot(ss, k_rels_o, label='oil')
plt.xlabel('S oil')
plt.ylabel("k relative")
plt.title('Relative permeability')
plt.legend()
plt.show()

In [ ]:
from reservoir import ResState

In [ ]:
p = ResState(p, p_0, prop)
s_o = ResState(s_o, s_b, prop)
s_w = ResState(s_w, 1 - s_b, prop)

Дальше немного сложенее - теперь нужно вводить межблочную относительную проницаемость

Небольшие изменения потерпит матарица, отвечающая за лапласиан

In [ ]:
from reservoir import get_lapl_one_ph

In [ ]:
lapl_w = torch.zeros((prop.nx * prop.ny, prop.nx * prop.ny), device=device, dtype=dtype)
lapl_o = torch.zeros((prop.nx * prop.ny, prop.nx * prop.ny), device=device, dtype=dtype)

In [ ]:
for i in tqdm(range(100)):
    # p.v = p.v.cpu().numpy()
    # s_o.v = s_o.v.cpu().numpy()
    lapl_w, si = get_lapl_one_ph(p=p, s=s_o, ph='o', prop=prop, device=device, dtype=dtype)
    # s_o.v = torch.tensor(s_o.v, device=device, dtype=dtype)
    # p.v = torch.tensor(p.v, device=device, dtype=dtype)

In [ ]:
type(s_o.v)

Также изменился вектор для граничных условий

In [ ]:
from reservoir import get_q_bound

In [ ]:
q_bound_w = torch.zeros((prop.nx * prop.ny, 1), device=device, dtype=dtype)
q_bound_o = torch.zeros((prop.nx * prop.ny, 1), device=device, dtype=dtype)

In [ ]:
get_q_bound(p, s_w, 'w', prop, q_bound_w)
get_q_bound(p, s_o, 'o', prop, q_bound_o)

Теперь про моделирование скважины

Закинем координаты скважин

In [ ]:
pos_r = {(12, 8): 0.1, (18, 21): 0.1}
delta_p_well = -100 * 6894.
delta_p_vec = torch.ones((nx*ny, 1), device=device, dtype=dtype) * delta_p_well

In [ ]:
from reservoir import get_j_matrix

In [ ]:
j_o = torch.zeros((prop.nx * prop.ny, 1), device=device, dtype=dtype)
j_w = torch.zeros((prop.nx * prop.ny, 1), device=device, dtype=dtype)
_ = get_j_matrix(p=p, s=s_o, pos_r=pos_r, ph='o', prop=prop, j_matr=j_o)
_ = get_j_matrix(p=p, s=s_w, pos_r=pos_r, ph='w', prop=prop, j_matr=j_w)

И последний штрих - перед производной по времени есть $V_b\big(S_o c_o + S_w c_w  +1\cdot c_r\big)$. Назовём это `dt_comp_sat`

Кстати, у нас теперь `2d` и теперь нужно делать `.reshape`

In [ ]:
nxny_ones = torch.ones((nx*ny, 1), device=device, dtype=dtype)
nxny_eye = torch.eye(nx*ny, device=device, dtype=dtype) 

In [ ]:
dt_comp_sat = dx * dy * d *(s_o.v * c_o+ s_w.v * c_w) # + nxny_ones * c_r)

Закаидываем пареметры для запуска сессии и записи истории

In [ ]:
n_iter = int(5e2)
p_ex = np.ones((nx, 1)) * p_0
t = 0
xs = list((np.linspace(0, nx-1, nx) + dx/2) * dx)
plot_freq = int(1e3)
times = []
p_well_hist = {}
s_o_well_hist = {}
q_o_hist = {}
q_w_hist = {}
for w in pos_r:
    p_well_hist[w] = []
    s_o_well_hist[w] = []
    q_o_hist[w] = []
    q_w_hist[w] = []

In [ ]:
%%time
for i in tqdm(range(1, n_iter+1)):
    # gonna increase time step
    # matrixes depends on saturation
    dt_comp_sat = (dx * dy * d *(s_o.v) * c_o + s_w.v * c_w + nxny_ones * c_r)
    if (type(p.v) == torch.Tensor):
        p.v = p.v.cpu().numpy()
    if (type(s_o.v) == torch.Tensor):
        s_o.v = s_o.v.cpu().numpy()
    if (type(s_w.v) == torch.Tensor):
        s_w.v = s_w.v.cpu().numpy()
    _ = get_j_matrix(p=p, s=s_o, pos_r=pos_r, ph='o', prop=prop, j_matr=j_o)
    _ = get_j_matrix(p=p, s=s_w, pos_r=pos_r, ph='w', prop=prop, j_matr=j_w)
    lapl_w, si_w = get_lapl_one_ph(p=p, s=s_w, ph='w', prop=prop, device=device, dtype=dtype)
    lapl_o, si_o = get_lapl_one_ph(p=p, s=s_o, ph='o', prop=prop, device=device, dtype=dtype)
    _ = get_q_bound(p, s_w, 'w', prop, q_b=q_bound_w)
    _ = get_q_bound(p, s_o, 'o', prop, q_b=q_bound_o)
    if (type(s_o.v) == np.ndarray):
        s_o.v = torch.tensor(s_o.v, device=device, dtype=dtype)
    if (type(s_w.v) == np.ndarray):
        s_w.v = torch.tensor(s_w.v, device=device, dtype=dtype)
    if (type(p.v) == np.ndarray):
        p.v = torch.tensor(p.v, device=device, dtype=dtype)
    prop.dt = 0.1 * 0.5 * phi * dt_comp_sat.min() / (si_o + si_w)
    # matrix for implicit pressure
    
    a = phi * nxny_eye * dt_comp_sat - (lapl_w + lapl_o) * prop.dt
    #right hand state for ax = b
    
    b = phi * dt_comp_sat * p.v + q_bound_w*prop.dt + q_bound_o*prop.dt
    
    b += (j_o * B_o + j_w * B_w) * delta_p_vec * prop.dt
    # upd time stamp
    t += prop.dt / 60 / 60 / 24
    # solve p
    p_new, _ = torch.solve(b, a)
    dp = p_new - p.v
    
    a = dx*dy*d * phi *(nxny_ones + (c_r + c_o) * (p_new - p.v))
    b = phi * dx*dy*d * s_o.v + prop.dt * (lapl_o.matmul(p_new) + q_bound_o + j_o * B_o * delta_p_vec)
    # upd target values
    s_o = ResState((b / a), s_b, prop) 
    s_w = ResState(nxny_ones - s_o.v, 1 - s_b, prop) 
    p = ResState(p_new, p_0, prop)
    if i % plot_freq == 0:
        
        if (type(p.v) == torch.Tensor):
            p.v = p.v.cpu().numpy()
        if (type(s_o.v) == torch.Tensor):
            s_o.v = s_o.v.cpu().numpy()
        if (type(s_w.v) == torch.Tensor):
            s_w.v = s_w.v.cpu().numpy()
        
        q_o = ((-1) * j_o * (delta_p_vec)).reshape((nx, ny))
        q_w = ((-1) * j_w * (delta_p_vec)).reshape((nx, ny))
        
        # gonna set wells as nan to see gradient
        # p_v_disp = p.v.reshape((nx, ny)).copy() / 6894.
        # s_o_disp = s_o.v.reshape((nx, ny)).copy()
        
        times.append(t)
        for w in pos_r:
            p_well_hist[w].append(p[w] / 6894.)
            s_o_well_hist[w].append(s_o[w])
            q_o_hist[w].append(q_o[w])
            q_w_hist[w].append(q_w[w])
            # set wells as nan to see gradient
            # p_v_disp[w] = np.nan
            # s_o_disp[w] = np.nan
        display.clear_output(wait=True)
        f, ax = plt.subplots(nrows=3, ncols=2, figsize=(16, 16))
        f.tight_layout(pad=6.0)
        
        sns.heatmap(p.v.reshape((nx, ny)) / 6894., ax=ax[0][0], cbar=True)
        ax[0][0].set_title(f'Pressure, psi\nt={t}, days\n nan at well pos')
        
        sns.heatmap(s_o.v.reshape((nx, ny)), ax=ax[0][1], cbar=True, fmt=".2f")
        ax[0][1].set_title(f'Saturation, oil, psi\nt={t}, days\n nan at well pos')
        
        for w in pos_r:
            ax[1][0].plot(times, p_well_hist[w] , label=f'{w}')
            ax[1][1].plot(times, s_o_well_hist[w] , label=f'{w}')
            ax[2][0].plot(times, q_o_hist[w] , label=f'{w}')
            ax[2][1].plot(times, q_w_hist[w] , label=f'{w}')
        ax[1][0].set_xlabel('time, days')
        ax[1][0].set_ylabel('pressure, psi')
        ax[1][0].set_title('Pressure in wells')
        ax[1][0].legend()
        
        ax[1][1].set_xlabel('time, days')
        ax[1][1].set_ylabel('saturation')
        ax[1][1].set_title('Oil saturation in wells')
        ax[1][1].legend()
        
        ax[2][0].set_xlabel('time, days')
        ax[2][0].set_ylabel('q, m3/s')
        ax[2][0].set_title('Oil rate')
        ax[2][0].legend()
        
        ax[2][1].set_xlabel('time, days')
        ax[2][1].set_ylabel('q, m3/s')
        ax[2][1].set_title('Water rate')
        ax[2][1].legend()
                
        plt.show()

GG, почемуто `torch` медленней

Посмотрим, где именно проседает, в инициализации, или умножении

In [ ]:
for i in tqdm(range(1, 100)):
    # gonna increase time step
    # matrixes depends on saturation
    dt_comp_sat = (dx * dy * d *(s_o.v) * c_o + s_w.v * c_w + nxny_ones * c_r)
    #lapl_w, si_w = get_lapl_one_ph(p=p, s=s_w, ph='w', prop=prop, device=device, dtype=dtype)
    #lapl_o, si_o = get_lapl_one_ph(p=p, s=s_o, ph='o', prop=prop, device=device, dtype=dtype)
    
    _ = get_j_matrix(p=p, s=s_o, pos_r=pos_r, ph='o', prop=prop, j_matr=j_o)
    _ = get_j_matrix(p=p, s=s_w, pos_r=pos_r, ph='w', prop=prop, j_matr=j_w)

In [ ]:
for i in tqdm(range(1, 1000)):
    a = phi * nxny_eye * dt_comp_sat - (lapl_w + lapl_o) * dt
    #right hand state for ax = b
    b = phi * dt_comp_sat * p.v + q_bound_w*dt + q_bound_o*dt 
    b += (j_o * B_o + j_w * B_w) * delta_p_vec * dt
    # upd time stamp
    t += dt
    # solve p
    p_new, _ = torch.solve(b, a)
    # dp = p_new - p.v
    a = dx*dy*d * phi *(nxny_ones + (c_r + c_o) * (p_new - p.v))
    b = phi * dx*dy*d * s_o.v + dt * (lapl_o.matmul(p_new) + q_bound_o + j_o * B_o * delta_p_vec)
    # upd target values
    s_o = ResState((b / a), s_b, prop) 
    s_w = ResState(nxny_ones - s_o.v, 1 - s_b, prop) 
    p = ResState(p_new, p_0, prop)

In [ ]:
for i in tqdm(range(1, 100)):
    dt_comp_sat = (dx * dy * d *(s_o.v) * c_o + s_w.v * c_w + nxny_ones * c_r)
    _ = get_j_matrix(p=p, s=s_o, pos_r=pos_r, ph='o', prop=prop, j_matr=j_o)
    _ = get_j_matrix(p=p, s=s_w, pos_r=pos_r, ph='w', prop=prop, j_matr=j_o)

In [ ]:
for i in tqdm(range(1, 15)):
    dt_comp_sat = (dx * dy * d *(s_o.v) * c_o + s_w.v * c_w + nxny_ones * c_r)
    _ = get_lapl_one_ph(p=p, s=s_w, ph='w', prop=prop, lapl=lapl_w)
    _ = get_lapl_one_ph(p=p, s=s_o, ph='o', prop=prop, lapl=lapl_o)

In [ ]:
for i in tqdm(range(1, 100)):
    dt_comp_sat = (dx * dy * d *(s_o.v) * c_o + s_w.v * c_w + nxny_ones * c_r)
    _ = get_q_bound(p, s_w, 'w', prop, q_b=q_bound_w)
    _ = get_q_bound(p, s_o, 'o', prop, q_b=q_bound_w)

In [ ]:
a = phi * nxny_eye * dt_comp_sat - (lapl_w + lapl_o) * dt
b = phi * dt_comp_sat * p.v + q_bound_w*dt + q_bound_o*dt 
b += (j_o * B_o + j_w * B_w) * delta_p_vec * dt
for i in tqdm(range(1, 1000)):
    p_new, _ = torch.solve(b, a)

In [ ]:
(625 * 625)

In [ ]:
((60 * 60 * 24 * 365) / 152) / 60 / 60 / 2 

In [ ]:
dx = 3
dy = 3
depth = 10

In [ ]:
25 * 3

In [ ]:
for i in tqdm(range(1, 1000)):
    a = dx*dy*d * phi *(nxny_ones + (c_r + c_o) * (p_new - p.v))
    b = phi * dx*dy*d * s_o.v + dt * (lapl_o.matmul(p_new) + q_bound_o + j_o * B_o * delta_p_vec)

In [ ]:
a = dx*dy*d * phi *(nxny_ones + (c_r + c_o) * (p_new - p.v))
b = phi * dx*dy*d * s_o.v + dt * (lapl_o.matmul(p_new) + q_bound_o + j_o * B_o * delta_p_vec)
for i in tqdm(range(1, 1000)):
    s_o = ResState((b / a), s_b, prop) 
    s_w = ResState(nxny_ones - s_o.v, 1 - s_b, prop) 
    p = ResState(p_new, p_0, prop)

Имеет смысл реально запариться с иницализацией

In [ ]:
1 / (1 / 41 + 1 / 72)

Инициализация матриц для подсчёта - самое долгое

In [ ]:
from reservoir import add_peice_of_lapl, get_lapl_one_ph_mth
from mult_func import test_glob_mp

In [ ]:
import multiprocessing
from multiprocessing import Array

In [ ]:
def mp_stuff(arr):
    if __name__ == "__main__":
        procs = 8   # Number of processes to create

        # Create a list of jobs and then iterate through
        # the number of processes appending each process to
        # the job list 
        jobs = []
        for i in range(procs):
            process = multiprocessing.Process(target=test_glob_mp, 
                                              args=(arr, i)
                                             )
            jobs.append(process)

        # Start the processes (i.e. calculate the random number lists)      
        for j in jobs:
            j.start()

        # Ensure all of the processes have finished
        for j in jobs:
            j.join()

In [ ]:
arrr = Array('d', (prop.nx * prop.ny))

In [ ]:
lapl = get_lapl_one_ph_mth(p=p, s=s_w, ph='w', prop=prop, n_th=8)

In [ ]:
lapl_w = get_lapl_one_ph(p=p, s=s_w, ph='w', prop=prop)

In [ ]:
(lapl_w[2]).max()

In [ ]:
for i in tqdm(range(1, n_iter)):
    lapl_w = get_lapl_one_ph(p=p, s=s_w, ph='w', prop=prop)

Начинаем потеть и подключаем torch

In [ ]:
import torch
import ctypes
ctypes.cdll.LoadLibrary('caffe2_nvrtc.dll')

Ну и будет ли это быстрее?

In [ ]:
A = torch.rand((700, 700), device=device)
B = torch.rand((700, 1), device=device)
X, _ = torch.solve(B, A)

In [ ]:
A

In [ ]:
A.shape

In [ ]:
a = np.random.rand(700, 700)
b = np.random.rand(700, 1)
for i in tqdm(range(1, n_iter)):
    sol = np.linalg.solve(a, b)

In [ ]:
A = torch.rand((700, 700), device=device)
B = torch.rand((700, 1), device=device)
for i in tqdm(range(1, n_iter)):
    X, _ = torch.solve(B, A)

Намного быстрее. А что с инициализацией?

In [ ]:
for i in tqdm(range(1, n_iter)):
    A = torch.rand((700, 700), device=device)
    B = torch.rand((700, 1), device=device)